# V2 F1 calculations
Add ribonanzanet preds with singlets and recompute F1 with updated predictions from other algorithms

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#get v2 data
df=pd.read_csv("RibonanzaSupplementalTablesV2_Table_S8.csv",skiprows=2)[:78]
df

,name,dataset,sequence,structure,length,num_pseudoknots,RibonanzaNet PRED,RibonanzaNet no pretraining PRED,Vienna2 PRED,CONTRAfold2 PRED,...,CONTRAfold2 (HK) F1,Shapify-Hfold F1,SPOT-RNA F1,E-value,log E-value,TM score,RMSD (RibonanzaNet) ↓,RMSD (SPOT-RNA) ↓,lDDT (RibonanzaNet) ↑,lDDT (SPOT-RNA) ↑
0,6MXQ,pdb,GGCAGUAUAGUCCGAACUGCAACUUCGGUUCACCUUCUCUCUAACUGCC,((((((.(((...(((.((.(((....)))))..)))...))))))))),49,0,((((((.(((.(((((.......)))))............))))))))),((((((.....(((((.......)))))...............)))))),((((((.(((...((((((......)))))).........))))))))),((((((.(((.(((((.......)))))............))))))))),...,0.444444,0.937500,0.428571,1.000000e+00,0.000000,0.5783,17.80,19.60,0.408,0.365
1,6NOA,pdb,GGCAGUAUAGUCCGAACUGCAAAUCUUAUUUUCUUUUCACCUUCUC...,((((((.(((...(((.((.(((...........)))))..))).....,56,0,((((((.(((.......................................,((((((..............((.......))..................,((((((.(((...(((.((.(((...........)))))..))).....,((((((.(((...(((.((..((..........))..))..))).....,...,0.640000,0.823529,0.692308,1.000000e+00,0.000000,0.5564,12.93,12.16,0.549,0.464
2,6PK9,pdb,GGAGGGUAGACUCGCUCUCC,(((((((......))))))),20,0,(((((((......))))))),(((((((......))))))),(((((((......))))))),(((((((......))))))),...,1.000000,1.000000,1.000000,1.000000e+00,0.000000,0.6317,3.73,3.46,0.743,0.725
3,6TB7,pdb,GGCUUCAACAACCCCGUAGGUUGGGCCGAAAGGCAGCGAAUCUACU...,(((((((.....[[.((((((((.(((....))).])]))))))))...,52,2,(((((((........(((((((..(((....)))....))))))))...,(((((((.................(((....)))...........)...,(((((((........(((((((.((((....)))..).))))))))...,(((((((........(((((((.((((....)))..).))))))))...,...,0.918919,0.918919,0.871795,1.000000e+00,0.000000,0.4747,7.24,6.79,0.625,0.617
4,7D7W,pdb,GCUUCAACAACCCCGUAGGUGGGGACGAAAGUCAGCGCACCUACUG...,((((((.....[[.((((((((.(((....))).])])))))))))...,51,2,((((((........(((((((..(((....)))....)))))))))...,((((((.....((..[[[[[.))(((....))).....]]]]].))...,((((((........(((((((.((((....)))..).)))))))))...,((((((........(((((((.((((....)))..).)))))))))...,...,0.689655,0.914286,0.842105,1.000000e+00,0.000000,0.5015,10.15,7.18,0.519,0.547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,R1138v1,casp_15,GGGAGAGUACUAUUCAGAUGCAGACCGCAAGUUCAGAGCGGUUUGC...,((((((((((((((((((((((.(((((..[[[[[[.))))).)))...,720,5,((((((((((((((((((((((((((((.........)))))))))...,(((((((((((.....((((((((((((.........)))))))))...,((((((((((((((((((((((((((((.........)))))))))...,((((((((((((((((((((((((((((.........)))))))))...,...,0.896194,0.867725,0.808511,1.000000e+00,0.000000,0.1959,44.55,45.74,0.274,0.123
74,R1149,casp_15,GGACACGAGUAACUCGUCUAUCUUCUGCAGGCUGCUUACGGUUUCG...,(((((..((...(((.(((((((((((..((((((.(((((........,124,0,(((((.......(((.(((((((((((..((((((..((((........,((((((((.....))).......................((........,(((((..((...(((.((((((((.....((((((.(((((........,(((((..((...(((.(((((((((((..((((((.(((((........,...,0.900000,0.939759,0.712329,6.640000e-60,-59.177832,0.4025,12.73,18.92,0.651,0.520
75,R1156,casp_15,GGAGCAUCGUGUCUCAAGUGCUUCACGGUCACAAUAUACCGUUUCG...,(((((.((.((.(((..(((((((((((((((...((.(((........,135,0,(((((.((.((.(((..(((((((((.(((((......(((........,(((((.((..................((([[......))).........,(((((.((.((.(((..((((((.(((((........)))))..((...,(((((.((.((.(((..((((((....(((((......(((........,...,0.600000,0.800000,0.632911,5.660000e-66,-65.247184,0.3901,28.56,20.45,0.554,0.470
76,R1189,casp_15,GCGUACAGGGAACACGCAACCCCGAAGGAUCGGGGAAGGGACGUCG...,(((...........)))..(((((......)))))........((....,118,0,((((.........))))..((((((....)))))).......((((...,(((...........)))...((((......))))...............,((((.........))))..((((((....))))))...(((.((((...,((((.........))))..((((((....))))))...(((.((((...,...,0.734694,0.698413,0.692308,1.000000e+00,0.000000,0.2798,33.08,31.72,0.485,0.487


In [3]:
#get ribonanzanet predictions
rn_pdb=pd.read_parquet("finetuned_test_ws.parquet").rename(columns={'Sequence_ID':'name'})
rn_casp=pd.read_csv("casp15_ribonanzanet_ws.csv").rename(columns={'sequence_id':'name'})

rn_preds=pd.concat([rn_pdb[['name','RibonanzaNet_Hungarian']],
                    rn_casp[['name','RibonanzaNet_Hungarian']]])

rn_preds=df[['name']].merge(rn_preds,how='left',on='name')

rn_preds


,name,RibonanzaNet_Hungarian
0,6MXQ,((((((.(((.(((((.......)))))............)))))))))
1,6NOA,((((((.(((....(...(.............).........)......
2,6PK9,(((((((.(...))))))))
3,6TB7,(((((((..(.....[[[[[[)..(((....))).....]]]]]])...
4,7D7W,((((((........((((((((.(((....)))..).)))))))))...
...,...,...
73,R1138v1,((((((((((((((((((((((((((((....[....)))))))))...
74,R1149,(((((.......(((.(((((((((((..((((((..((((........
75,R1156,(((((.((.((.(((..(((((((((((((((......(((........
76,R1189,((((.........))))..((((((....))))))....((.((((...


In [4]:
df['RibonanzaNet PRED']=rn_preds['RibonanzaNet_Hungarian']

In [5]:
rn_casp.columns

Index(['name', 'sequence', 'structure', 'bp', 'pairs', 'RNA_sequence',
       'RibonanzaNet_Hungarian', 'RibonanzaNet_Hungarian_bps',
       'RibonanzaNet_Hungarian_F1', 'RibonanzaNet_Hungarian_CP_F1'],
      dtype='object')

In [6]:
df.columns

Index(['name', 'dataset', 'sequence', 'structure', 'length', 'num_pseudoknots',
       'RibonanzaNet PRED', 'RibonanzaNet no pretraining PRED', 'Vienna2 PRED',
       'CONTRAfold2 PRED', 'EternaFold PRED', 'IPknot PRED', 'Knotty PRED',
       'Vienna2 (TK) PRED', 'Vienna2 (HK) PRED', 'EternaFold ThreshKnot PRED',
       'EternaFold (HK) PRED', 'CONTRAfold2 ThreshKnot PRED',
       'CONTRAfold2 (HK) PRED', 'Shapify-Hfold PRED', 'SPOT-RNA PRED',
       'RibonanzaNet F1', 'RibonanzaNet no pretraining F1', 'Vienna2 F1',
       'CONTRAfold2 F1', 'EternaFold F1', 'IPknot F1', 'Knotty F1',
       'Vienna2 (TK) F1', 'Vienna2 (HK) F1', 'EternaFold ThreshKnot F1',
       'EternaFold (HK) F1', 'CONTRAfold2 ThreshKnot F1',
       'CONTRAfold2 (HK) F1', 'Shapify-Hfold F1', 'SPOT-RNA F1', 'E-value',
       'log E-value', 'TM score', 'RMSD (RibonanzaNet) ↓', 'RMSD (SPOT-RNA) ↓',
       'lDDT (RibonanzaNet) ↑', 'lDDT (SPOT-RNA) ↑'],
      dtype='object')

In [7]:
df.columns[6:21]

Index(['RibonanzaNet PRED', 'RibonanzaNet no pretraining PRED', 'Vienna2 PRED',
       'CONTRAfold2 PRED', 'EternaFold PRED', 'IPknot PRED', 'Knotty PRED',
       'Vienna2 (TK) PRED', 'Vienna2 (HK) PRED', 'EternaFold ThreshKnot PRED',
       'EternaFold (HK) PRED', 'CONTRAfold2 ThreshKnot PRED',
       'CONTRAfold2 (HK) PRED', 'Shapify-Hfold PRED', 'SPOT-RNA PRED'],
      dtype='object')

In [8]:
from ast import literal_eval
from utils import *

def get_stats(df,col):

    F1s=[]
    crossed_pair_F1s=[]
    for true_db, predicted_db in zip(df['structure'],df[col]):
        #predicted_bp=dotbrackte2bp(predicted_bp)

        if predicted_db!=predicted_db: # for edge case of Shapify-Hfold PRED where there is an nan as pred
            F1s.append(0)
            crossed_pair_F1s.append(0)


        else:
            true_bp=dotbrackte2bp(true_db)
            predicted_bp=dotbrackte2bp(predicted_db)



            crossed_pairs,crossed_pairs_set=detect_crossed_pairs(true_bp)
            predicted_crossed_pairs,predicted_crossed_pairs_set=detect_crossed_pairs(predicted_bp)
            
            _,_,f1=calculate_f1_score_with_pseudoknots(true_bp, predicted_bp)
            F1s.append(f1)
            

            if len(crossed_pairs)>0:
                _,_,crossed_pair_f1=calculate_f1_score_with_pseudoknots(crossed_pairs, predicted_crossed_pairs)
                crossed_pair_F1s.append(crossed_pair_f1)
            elif len(crossed_pairs)==0 and len(predicted_crossed_pairs)>0:
                crossed_pair_F1s.append(0)
            else:
                crossed_pair_F1s.append(np.nan)
        

        

    F1s=np.array(F1s)
    crossed_pair_F1s=np.array(crossed_pair_F1s)

    print(f"for {col}")
    print('global F1 mean',np.mean(F1s))
    print('global F1 median',np.median(F1s))
    print('crossed pair F1 mean',np.nanmean(crossed_pair_F1s))
    print('crossed pair F1 median',np.nanmedian(crossed_pair_F1s))

    return F1s, crossed_pair_F1s

In [9]:
all_symbols=set()
for col in df.columns[6:21]:
    print(col)
    F1s, crossed_pair_F1s=get_stats(df,col)

    model_name=col[:-5]

    df[f'{model_name} F1']=F1s
    df[f'{model_name} crossed-pair F1']=crossed_pair_F1s
    for s in df[col]:
        for c in set(s):
            all_symbols.add(c)


    # print((df[f'{model_name} F1'].values.astype('float32')-F1s).max())
    # print((df[f'{model_name} F1'].values.astype('float32')-F1s).mean())

RibonanzaNet PRED
for RibonanzaNet PRED
global F1 mean 0.8907149704648699
global F1 median 0.9212962962962963
crossed pair F1 mean 0.5365891720416737
crossed pair F1 median 0.497536108751062
RibonanzaNet no pretraining PRED
for RibonanzaNet no pretraining PRED
global F1 mean 0.6687656433545658
global F1 median 0.6838432635534085
crossed pair F1 mean 0.13269815190886836
crossed pair F1 median 0.0
Vienna2 PRED
for Vienna2 PRED
global F1 mean 0.8125474524184927
global F1 median 0.8918128654970761
crossed pair F1 mean 0.0
crossed pair F1 median 0.0
CONTRAfold2 PRED
for CONTRAfold2 PRED
global F1 mean 0.8147764239644696
global F1 median 0.8663615560640732
crossed pair F1 mean 0.0
crossed pair F1 median 0.0
EternaFold PRED
for EternaFold PRED
global F1 mean 0.8114209967605301
global F1 median 0.8709528214616096
crossed pair F1 mean 0.0
crossed pair F1 median 0.0
IPknot PRED
for IPknot PRED
global F1 mean 0.8067103844232347
global F1 median 0.838034722424172
crossed pair F1 mean 0.08951476389

In [14]:
for s in df['structure']:
    for c in set(s):
        all_symbols.add(c)

In [15]:
all_symbols # check all symbols ignore x ok?

{'(',
 ')',
 '.',
 '<',
 '>',
 'A',
 'B',
 'C',
 '[',
 ']',
 'a',
 'b',
 'c',
 'x',
 '{',
 '}'}

# get ATOM 1 clusters and score pdb

Get rid of duplicate sequence/structure \
Get rid of no-structure sequences (i.e. all '.' in structure)

In [11]:
atom1_split=pd.read_csv("pdb_2d_split.csv")
atom1_split=atom1_split[atom1_split['train_or_test']=='test'].reset_index(drop=True)
atom1_split=atom1_split.drop_duplicates('sequence')
atom1_split=atom1_split.drop_duplicates('structure')

filter_no_structure=[]
for dbn in atom1_split['structure']:
    if set(list(dbn))==set('.'):
        filter_no_structure.append(False)
    else:
        filter_no_structure.append(True)
atom1_split=atom1_split[filter_no_structure].reset_index(drop=True)

atom1_split['name']=atom1_split['pdb_id']
atom1_split=df[['name']].merge(atom1_split[['name','test_cluster_idx']],how='inner',on='name')

atom1_split

,name,test_cluster_idx
0,6MXQ,0.0
1,6NOA,0.0
2,6PK9,1.0
3,6TB7,2.0
4,7D7W,2.0
...,...,...
61,8I43,53.0
62,8S95,54.0
63,8SP9,54.0
64,8SCF,55.0


In [52]:
#group and compute scores for PDB set

pdb_df=df[df['dataset']=='pdb'].reset_index(drop=True)
pdb_df['test_cluster_idx']=atom1_split['test_cluster_idx'].astype('int')

summary_df=pd.DataFrame()


avg_F1s=[]
med_F1s=[]
avg_CP_F1s=[]
med_CP_F1s=[]
model_names=[]
for col in df.columns[6:21]:
#for col in [df.columns[6:21].values[0]]:
#for col in ['RibonanzaNet no pretraining PRED']:

    model=col[:-5]
    model_names.append(model)
    grouped_f1s=[]
    grouped_CP_f1s=[]
    for g in pdb_df.groupby('test_cluster_idx'):
        scores=g[1][f'{model} F1'].values.mean()
        grouped_f1s.append(scores)
        grouped_CP_f1s.append(np.nanmean(g[1][f'{model} crossed-pair F1'].values))

        #print(g[1][f'{model} crossed-pair F1'])


    # print(model)
    # print(np.mean(grouped_f1s))

    avg_F1s.append(np.mean(grouped_f1s))
    med_F1s.append(np.median(grouped_f1s))
    avg_CP_F1s.append(np.nanmean(grouped_CP_f1s))
    med_CP_F1s.append(np.nanmedian(grouped_CP_f1s))

    # summary_df[f'{model} avg F1']=[np.mean(grouped_f1s)]
    # summary_df[f'{model} avg F1']=[np.mean(grouped_f1s)]

summary_df['Model']=model_names
summary_df['PDB Mean F1']=avg_F1s
summary_df['PDB Median F1']=med_F1s
summary_df['PDB Mean Crossed-pair F1']=avg_CP_F1s
summary_df['PDB Median Crossed-pair F1']=med_CP_F1s

summary_df

/tmp/ipykernel_8222/200322125.py:25: RuntimeWarning: Mean of empty slice
  grouped_CP_f1s.append(np.nanmean(g[1][f'{model} crossed-pair F1'].values))


,Model,PDB Mean F1,PDB Median F1,PDB Mean Crossed-pair F1,PDB Median Crossed-pair F1
0,RibonanzaNet,0.890493,0.933333,0.493570,0.406593
1,RibonanzaNet no pretraining,0.709938,0.733333,0.086860,0.000000
2,Vienna2,0.847690,0.928571,0.000000,0.000000
3,CONTRAfold2,0.830854,0.878049,0.000000,0.000000
4,EternaFold,0.831104,0.906667,0.000000,0.000000
5,IPknot,0.817640,0.864865,0.049707,0.000000
6,Knotty,0.831044,0.923077,0.141112,0.000000
7,Vienna2 (TK),0.861117,0.909091,0.000000,0.000000
8,Vienna2 (HK),0.863660,0.909091,0.000436,0.000000
9,EternaFold ThreshKnot,0.831158,0.909091,0.000000,0.000000


array(['pdb', 'casp_15'], dtype=object)

In [54]:
#group and compute scores for CASP set

pdb_df=df[df['dataset']=='casp_15'].reset_index(drop=True)
pdb_df['test_cluster_idx']=np.arange(len(pdb_df))


avg_F1s=[]
med_F1s=[]
avg_CP_F1s=[]
med_CP_F1s=[]
model_names=[]
for col in df.columns[6:21]:
#for col in [df.columns[6:21].values[0]]:
#for col in ['RibonanzaNet no pretraining PRED']:

    model=col[:-5]
    model_names.append(model)
    grouped_f1s=[]
    grouped_CP_f1s=[]
    for g in pdb_df.groupby('test_cluster_idx'):
        scores=g[1][f'{model} F1'].values.mean()
        grouped_f1s.append(scores)
        grouped_CP_f1s.append(np.nanmean(g[1][f'{model} crossed-pair F1'].values))


    avg_F1s.append(np.mean(grouped_f1s))
    med_F1s.append(np.median(grouped_f1s))
    avg_CP_F1s.append(np.nanmean(grouped_CP_f1s))
    med_CP_F1s.append(np.nanmedian(grouped_CP_f1s))

    # summary_df[f'{model} avg F1']=[np.mean(grouped_f1s)]
    # summary_df[f'{model} avg F1']=[np.mean(grouped_f1s)]

summary_df['Model']=model_names
summary_df['CASP Mean F1']=avg_F1s
summary_df['CASP Median F1']=med_F1s
summary_df['CASP Mean Crossed-pair F1']=avg_CP_F1s
summary_df['CASP Median Crossed-pair F1']=med_CP_F1s

summary_df

/tmp/ipykernel_8222/623332424.py:23: RuntimeWarning: Mean of empty slice
  grouped_CP_f1s.append(np.nanmean(g[1][f'{model} crossed-pair F1'].values))


,Model,PDB Mean F1,PDB Median F1,PDB Mean Crossed-pair F1,PDB Median Crossed-pair F1,CASP Mean F1,CASP Median F1,CASP Mean Crossed-pair F1,CASP Median Crossed-pair F1
0,RibonanzaNet,0.890493,0.933333,0.493570,0.406593,0.936915,0.957121,0.737252,0.896061
1,RibonanzaNet no pretraining,0.709938,0.733333,0.086860,0.000000,0.496751,0.488953,0.213056,0.153846
2,Vienna2,0.847690,0.928571,0.000000,0.000000,0.715532,0.820759,0.000000,0.000000
3,CONTRAfold2,0.830854,0.878049,0.000000,0.000000,0.822726,0.895047,0.000000,0.000000
4,EternaFold,0.831104,0.906667,0.000000,0.000000,0.804679,0.884423,0.000000,0.000000
5,IPknot,0.817640,0.864865,0.049707,0.000000,0.812538,0.886305,0.130819,0.000000
6,Knotty,0.831044,0.923077,0.141112,0.000000,0.681653,0.873575,0.252742,0.000000
7,Vienna2 (TK),0.861117,0.909091,0.000000,0.000000,0.780888,0.839236,0.000000,0.000000
8,Vienna2 (HK),0.863660,0.909091,0.000436,0.000000,0.790115,0.839236,0.000000,0.000000
9,EternaFold ThreshKnot,0.831158,0.909091,0.000000,0.000000,0.826724,0.911111,0.000000,0.000000


In [55]:
pdb_df

,name,dataset,sequence,structure,length,num_pseudoknots,RibonanzaNet PRED,RibonanzaNet no pretraining PRED,Vienna2 PRED,CONTRAfold2 PRED,...,Knotty crossed-pair F1,Vienna2 (TK) crossed-pair F1,Vienna2 (HK) crossed-pair F1,EternaFold ThreshKnot crossed-pair F1,EternaFold (HK) crossed-pair F1,CONTRAfold2 ThreshKnot crossed-pair F1,CONTRAfold2 (HK) crossed-pair F1,Shapify-Hfold crossed-pair F1,SPOT-RNA crossed-pair F1,test_cluster_idx
0,R1107,casp_15,GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUU...,.[[[[[...((((((((((.......))).]]]]]...(((((......,69,1,((((((...[[[[[[[(((.......))).))))))..(((((......,(((......[[.[[............[[[...)))....((((......,.........((((((((((....)))(((((((...((((.....)...,.........(((((((.((....))((((((((...((..((.......,...,0.378378,0.0,0.0,0.0,0.0,0.0,0.0,0.727273,0.733333,0
1,R1108,casp_15,GGGGGCCACAGCAGAAGCGUUCACGUCGCGGCCCCUGUCAGCCAUU...,[[[[[[...((((((((((.......))).]]]]]]..(((((......,69,1,(((((((..[[[[[[[(((.......))))))))))..(((((......,((((((...[[.[[................))))))...((((......,(((((((.(.((.(((...)))..)).).)))))))(.(((((......,(((((((.........(((.......))))))))))..(((((......,...,0.962963,0.0,0.0,0.0,0.0,0.0,0.0,0.962963,0.962963,1
2,R1116,casp_15,CGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGG...,.(((((((..((((.....[..)))).(((((((((((((((((((...,157,1,((((((((..((((.....[..)))).(((((((((((((((((((...,((((((((..((((........))))...((((((((............,((((((((..((((........)))).((((((((((((.((((((...,((((((((..((((.(....).)))).(((((((((((((((((((...,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2
3,R1117,casp_15,UUGGGUUCCCUCACCCCAAUCAUAAAAAGG,.((((...[[[...)))).........]]],30,1,(((((...[[[...)))))........]]],((..((.[[...))...)).........]],..((((......))))..............,((((((......).)))))...........,...,0.933333,0.0,0.0,0.0,0.0,0.0,0.0,0.933333,0.933333,3
4,R1126,casp_15,GGAAUCUCGCCCGAUGUUCGCAUCGGGAUUUGCAGGUCCAUGGAUU...,((((((((((((((((....)))))))((((.(((((((((((......,363,2,((((((((((((((((....)))))))((((((((((((((((......,((((((((.(((((((....))))))).((([[[[[((<<<<)).....,((((((((((((((((....)))))))((((((((((((((((......,((((((((((((((((....)))))))((((((((((((((((......,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.125000,4
5,R1128,casp_15,GGAAUAUCGUCAUGGUGAUUCGUCACCAUGAGGCUAGAUCUCAUAU...,((((((((((((((((((....)))))))))(((((((([[[[[[(...,238,2,((((((((((((((((((....)))))))))(((((((([[[[[[(...,...((.[[((....))((....)).........................,((((((((((((((((((....)))))))))((((((((......(...,((((((((((((((((((....)))))))))((((((((......(...,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.777778,5
6,R1136,casp_15,GGAUACGUCUACGCUCAGUGACGGACUCUCUUCGGAGAGUCUGACA...,((((((((((((((((((((((((((((((....)))))))))(((...,374,2,((((((((((((((((((((((((((((((....)))))))))(((...,(((((((.((............((((((((....)))))))).......,((((((((((((((((((((((((((((((....)))))))))(((...,((((((((((((((((((((((((((((((....)))))))))(((...,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.589474,6
7,R1138v1,casp_15,GGGAGAGUACUAUUCAGAUGCAGACCGCAAGUUCAGAGCGGUUUGC...,((((((((((((((((((((((.(((((..[[[[[[.))))).)))...,720,5,((((((((((((((((((((((((((((....[....)))))))))...,(((((((((((.....((((((((((((.........)))))))))...,((((((((((((((((((((((((((((.........)))))))))...,((((((((((((((((((((((((((((.........)))))))))...,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.596923,7
8,R1149,casp_15,GGACACGAGUAACUCGUCUAUCUUCUGCAGGCUGCUUACGGUUUCG...,(((((..((...(((.(((((((((((..((((((.(((((........,124,0,(((((.......(((.(((((((((((..((((((..((((........,((((((((.....))).......................((........,(((((..((...(((.((((((((.....((((((.(((((........,(((((..((...(((.(((((((((((..((((((.(((((........,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
9,R1156,casp_15,GGAGCAUCGUGUCUCAAGUGCUUCACGGUCACAAUAUACCGUUUCG...,(((((.((.((.(((..(((((((((((((((...((.(((........,135,0,(((((.((.((.(((..(((((((((((((((......(((........,(((((.((..................((([[......))).........,(((((.((.((.(((..((((((.(((((........)))))..((...,(((((.((.((

In [64]:
#group and compute scores for pdb+CASP set

pdb_df=df
pdb_df['test_cluster_idx']=1
pdb_df.loc[:66,'test_cluster_idx']=atom1_split['test_cluster_idx']
pdb_df.loc[66:,'test_cluster_idx']=np.arange(12)+100

pdb_df['test_cluster_idx']=pdb_df['test_cluster_idx'].astype('int')

avg_F1s=[]
med_F1s=[]
avg_CP_F1s=[]
med_CP_F1s=[]
model_names=[]
for col in df.columns[6:21]:
#for col in [df.columns[6:21].values[0]]:
#for col in ['RibonanzaNet no pretraining PRED']:

    model=col[:-5]
    model_names.append(model)
    grouped_f1s=[]
    grouped_CP_f1s=[]
    for g in pdb_df.groupby('test_cluster_idx'):
        scores=g[1][f'{model} F1'].values.mean()
        grouped_f1s.append(scores)
        grouped_CP_f1s.append(np.nanmean(g[1][f'{model} crossed-pair F1'].values))


    avg_F1s.append(np.mean(grouped_f1s))
    med_F1s.append(np.median(grouped_f1s))
    avg_CP_F1s.append(np.nanmean(grouped_CP_f1s))
    med_CP_F1s.append(np.nanmedian(grouped_CP_f1s))

    # summary_df[f'{model} avg F1']=[np.mean(grouped_f1s)]
    # summary_df[f'{model} avg F1']=[np.mean(grouped_f1s)]

summary_df['Model']=model_names
summary_df['PDB+CASP Mean F1']=avg_F1s
summary_df['PDB+CASP Median F1']=med_F1s
summary_df['PDB+CASP Mean Crossed-pair F1']=avg_CP_F1s
summary_df['PDB+CASP Median Crossed-pair F1']=med_CP_F1s

summary_df

/tmp/ipykernel_8222/2957315072.py:26: RuntimeWarning: Mean of empty slice
  grouped_CP_f1s.append(np.nanmean(g[1][f'{model} crossed-pair F1'].values))


,Model,PDB Mean F1,PDB Median F1,PDB Mean Crossed-pair F1,PDB Median Crossed-pair F1,CASP Mean F1,CASP Median F1,CASP Mean Crossed-pair F1,CASP Median Crossed-pair F1,PDB+CASP Mean F1,PDB+CASP Median F1,PDB+CASP Mean Crossed-pair F1,PDB+CASP Median Crossed-pair F1
0,RibonanzaNet,0.890493,0.933333,0.493570,0.406593,0.936915,0.957121,0.737252,0.896061,0.899063,0.941176,0.572177,0.777778
1,RibonanzaNet no pretraining,0.709938,0.733333,0.086860,0.000000,0.496751,0.488953,0.213056,0.153846,0.670580,0.703297,0.124719,0.000000
2,Vienna2,0.847690,0.928571,0.000000,0.000000,0.715532,0.820759,0.000000,0.000000,0.823292,0.913043,0.000000,0.000000
3,CONTRAfold2,0.830854,0.878049,0.000000,0.000000,0.822726,0.895047,0.000000,0.000000,0.829353,0.891813,0.000000,0.000000
4,EternaFold,0.831104,0.906667,0.000000,0.000000,0.804679,0.884423,0.000000,0.000000,0.826226,0.899281,0.000000,0.000000
5,IPknot,0.817640,0.864865,0.049707,0.000000,0.812538,0.886305,0.130819,0.000000,0.816698,0.883721,0.072882,0.000000
6,Knotty,0.831044,0.923077,0.141112,0.000000,0.681653,0.873575,0.252742,0.000000,0.803464,0.916602,0.176993,0.000000
7,Vienna2 (TK),0.861117,0.909091,0.000000,0.000000,0.780888,0.839236,0.000000,0.000000,0.846306,0.909091,0.000000,0.000000
8,Vienna2 (HK),0.863660,0.909091,0.000436,0.000000,0.790115,0.839236,0.000000,0.000000,0.850083,0.909091,0.000302,0.000000
9,EternaFold ThreshKnot,0.831158,0.909091,0.000000,0.000000,0.826724,0.911111,0.000000,0.000000,0.830339,0.911111,0.000000,0.000000


In [65]:
!mkdir output

summary_df.to_csv('output/replace_TableS7.csv',index=False)
df.to_csv('output/replace_TableS8.csv',index=False)